In [1]:
!pip install transformers newspaper3k torch lxml-html-clean --quiet

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 60.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
from transformers import pipeline
from newspaper import Article
import requests

from bs4 import BeautifulSoup

import random

from IPython.display import Markdown, display

In [114]:
# 1. PlannerAgent
class PlannerAgent:

    def decide_topic(self):

        # grown-up topics
        topics = ["Artificial Intelligence", "Space Exploration", "Climate Change", "Cryptocurrency", "Finance", "Weather", "Politics"]

        # fun topics - experimenting with these
        # topics = ["Florida Gators Football Recruiting", "Texas Longhorns Recruiting", "NBA News", "Major League Baseball News", "NFL News"]

        return random.choice(topics)


# 2. SearcherAgent
class SearcherAgent:

    def __init__(self):
        self.headers = {'User-Agent': 'Mozilla/5.0'}

    def search_articles(self, topic, max_articles=3):

        query = topic.replace(" ", "+")

        url = f"https://www.google.com/search?q={query}&tbm=nws"

        res = requests.get(url, headers=self.headers)

        soup = BeautifulSoup(res.text, "html.parser")

        links = []

        for a in soup.select('a'):

            href = a.get('href')

            if href and "http" in href and "google.com" not in href:

                if len(links) < max_articles:

                    links.append(href.split("&")[0].replace("/url?q=", ""))

        return links


# 3. SummarizerAgent
class SummarizerAgent:

    def __init__(self):
        self.summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

    def summarize_article(self, url):

        try:

            article = Article(url)
            article.download()
            article.parse()
            text = article.text[:1024]  # limit to model input size, (1024)
            summary = self.summarizer(text, max_length=100, min_length=30, do_sample=False)[0]['summary_text']

            return article.title, summary, url

        except Exception as e:

            return "Failed to load", str(e), url


# 4. CompilerAgent
class CompilerAgent:

    def compile_newsletter(self, topic, summaries):

        response = f"# Hey Alex! Here's what's going on today in the world of {topic}\n\n"

        for i, (title, summary, url) in enumerate(summaries, 1):

            # gives #th and article title
            response += f"## {i}. {title}\n"

            # gives summary blurb
            response += f"{summary}\n\n"

            # 'read more' link
            response += f"[Read more]({url})\n\n---\n"

        return response


# runs an 'Agentic' style workflow

planner = PlannerAgent()

searcher = SearcherAgent()

summarizer = SummarizerAgent()

compiler = CompilerAgent()


topic = planner.decide_topic()
print(f"* PlannerAgent chose topic: {topic}.")

urls = searcher.search_articles(topic)
print(f"* SearcherAgent grabbed {len(urls)} articles.")

summaries = [summarizer.summarize_article(url) for url in urls]

newsletter = compiler.compile_newsletter(topic, summaries)

display(Markdown(newsletter))

Device set to use cpu


* PlannerAgent chose topic: Space Exploration.
* SearcherAgent grabbed 3 articles.


# Hey Alex! Here's what's going on today in the world of Space Exploration

## 1. SpaceX CRS-32 Dragon cargo capsule arrives at the ISS with 6,700 pounds of supplies (video)
A SpaceX Dragon cargo capsule has arrived at the International Space Station (ISS) this morning (April 22), wrapping up about 28 hours traveling on orbit. Dragon will deliver about 6,700 pounds (3,040 kilograms) of supplies to the astronauts currently living and working on the orbiting lab.

[Read more](https://www.space.com/space-exploration/international-space-station/watch-spacex-dragon-cargo-capsule-arrive-at-the-iss-today)

---
## 2. It's alive! It's alive! Orion throws back its cover | Space picture of the day for April 22, 2025
The Orion Crew Module underwent a jettison test of its forward bay cover at NASA’s Neil Armstrong Test Facility in Sandusky, Ohio. The cover is the last component of the spacecraft that must eject before parachutes deploy to ensure a safe landing.

[Read more](https://www.space.com/space-exploration/artemis/its-alive-its-alive-orion-throws-back-its-cover-space-picture-of-the-day-for-april-22-2025)

---
## 3. The Space Review: “A bonafide frigging flight”: How NS-31 broke spaceflight norms and created an online uproar
The six women who flew on Blue Origin’s New Shepard NS-31 mission April 14. Kerianne Flynn, Katy Perry, Lauren Sánchez, Aisha Bowe, Gayle King, and Amanda Nguyen.

[Read more](https://www.thespacereview.com/article/4975/1)

---
